# Two Dimensional Pooling Layers

- toc: true
- badges: true
- comments: false
- categories: [jax, pooling]
- hide: false

## Introduction


## Purpose of Pooling Layers

## Import Libraries

For now, I only need numpy and tensorflow.

In [4]:
import numpy as np
import tensorflow as tf

## Implementation from First Principles

In [101]:
def downsample_images(input_batch, pool_size, strides, pool_fn=np.max):
    batch_size, xm, xn, num_channels  = input_batch.shape 
    
    pm, pn = pool_size 
    sm, sn = strides
    
    ym, yn = 1 + ((xm - pm)//sm), 1 + ((xn - pn)//sn) 
    y = np.zeros((batch_size, ym, yn, num_channels))

    for iy, ix in enumerate(range(0, xm-pm+1, sm)):
        for jy, jx in enumerate(range(0, xn-pn+1, sn)):
            y[:, iy, jy, :] = pool_fn(
                input_batch[:, ix:ix+pm, jx:jx+pn, :],
                axis=(1,2)
            )         
    return y    

## Compare to Keras

In [102]:
layer_keras = tf.keras.layers.MaxPool2D( 
    pool_size=(4, 4), 
    strides=(2,2),
    padding='valid'
)

In [103]:
input_batch = np.random.randn(32,28,28,3)

In [104]:
output_batch_keras = layer_keras(input_batch)

In [107]:
output_batch_numpy = downsample_images(
    input_batch, 
    pool_size=layer_keras.pool_size,
    strides=layer_keras.strides,
    pool_fn = np.max
)

To check that the Keras output and numpy outputs are approximately equal, I make sure that the absolute error is below a threshold.

In [109]:
assert np.max(np.abs(output_batch_keras - output_batch_numpy)) < 1e-16

Although the outputs are about the same, the Keras version runs much faster, as the following benchmarks show.    

In [110]:
%%timeit -n 100
downsample_images(
    input_batch, 
    pool_size=layer_keras.pool_size,
    strides=layer_keras.strides,
)

3.61 ms ± 47.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%%timeit -n 100
layer_keras(input_batch)

666 µs ± 52.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Pooling Layer in JAX

In [112]:
import jax
import jax.numpy as jnp
from typing import Tuple, List, Callable
from fastcore.basics import patch

In [116]:
class MaxPool2D: 
    pool_shape: Tuple[int,int]
    strides: Tuple[int,int]
    padding: str

#### Constructor


In [124]:
@patch
def __init__(
    self: MaxPool2D, 
    pool_shape=(2,2), 
    strides=(1,1), 
    padding='valid'):
    
    self.pool_shape = pool_shape
    self.strides = strides 
    self.padding = padding
         

#### `__call__` Method

In [125]:
@patch
def __call__(self: MaxPool2D, batch: jnp.ndarray):
    
    pool_shape = (1, ) + self.pool_shape + (1, )
    strides = (1, ) + self.strides + (1, ) 
    outputs = jax.lax.reduce_window(
        batch,
        -jnp.inf,
        jax.lax.max,
        window_dimensions=pool_shape,
        window_strides=strides,
        padding=self.padding
    )

    return outputs 

#### Adding to pytree Registry
 

In [128]:
@patch
def tree_flatten(self: MaxPool2D) -> Tuple[List[jnp.ndarray], dict]:
    params = (None,)
    metadata = {k: v for k, v in self.__dict__.items()}
    return params, metadata

In [129]:
@patch(cls_method=True)
def tree_unflatten(cls: MaxPool2D, metadata: dict, params:List[jnp.ndarray]):
    # This assumes that each key-value pair in the metadata dict corresponds to a constructor argument.
    layer = cls(**metadata)
    return layer

In [131]:
_ = jax.tree_util.register_pytree_node_class(MaxPool2D)

ValueError: Duplicate custom PyTreeDef type registration for <class '__main__.MaxPool2D'>.

## Compare JAX and Keras

In [133]:
layer_jax = MaxPool2D(
    pool_shape=layer_keras.pool_size,
    strides=layer_keras.strides,
    padding=layer_keras.padding
)

output_batch_jax = layer_jax(input_batch)

Because the following assertion passes, we can be reassured that both layers are calculating the same result.

In [134]:
assert np.max(np.abs(output_batch_keras - output_batch_jax)) < 1e-6

What about the calculation time?

In [137]:
%%timeit -n 100
layer_keras(input_batch)

659 µs ± 41.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
%%timeit -n 100
layer_jax(input_batch)

250 µs ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Conclusion